In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/CovidPrediction/

/content/drive/MyDrive/CovidPrediction


# Preprocessing Data
We need the following Observations:
- number of deaths (from Deaths_over_time.csv)
- number of tests (test_pos_over_time.csv, test_pos_over_time_antigen.csv)
- number of hospitalizations (newly_admitted_over_time.csv)
- number of sero positives (?)

Create Dataframes

In [4]:
df_deaths = pd.read_csv('data/overvaagningsdata-covid19-17052021-hb41/Deaths_over_time.csv', sep=';', thousands='.', decimal=',')
df_deaths.rename(columns={'Dato':'Date', 'Antal_døde':'Number_of_deaths'}, inplace=True)
df_deaths.drop(df_deaths.tail(1).index,inplace=True) # drop last row
df_deaths.tail(3)

,Date,Number_of_deaths
430,2021-05-15,0
431,2021-05-16,1
432,2021-05-17,0


In [5]:
df_tests_pcr = pd.read_csv('data/overvaagningsdata-covid19-17052021-hb41/Test_pos_over_time.csv', sep=';', thousands='.', decimal=',')
df_tests_pcr.drop(df_tests_pcr.tail(2).index,inplace=True) # drop last 2 rows
df_tests_pcr.drop(columns={'NotPrevPos', 'PrevPos', 'Tested_kumulativ'}, inplace=True)
df_tests_pcr.tail(3)

,Date,NewPositive,PosPct,Tested
473,2021-05-14,875,0.5,167338
474,2021-05-15,936,0.7,139267
475,2021-05-16,275,0.6,49910


In [6]:
df_tests_antigen = pd.read_csv('data/overvaagningsdata-covid19-17052021-hb41/Test_pos_over_time_antigen.csv', sep=';', thousands='.', decimal=',')
df_tests_antigen.drop(df_tests_antigen.tail(2).index,inplace=True) # drop last 2 rows
df_tests_antigen.drop(columns={'NotPrevPos', 'PrevPos', 'Tested_kumulativ'}, inplace=True)
df_tests_antigen.tail(3)

,Date,NewPositive,PosPct,Tested
401,2021-05-14,365,0.1,322404
402,2021-05-15,302,0.1,263018
403,2021-05-16,341,0.1,304357


In [7]:
df_hospitalizations = pd.read_csv('data/overvaagningsdata-covid19-17052021-hb41/Newly_admitted_over_time.csv', sep=';', thousands='.', decimal=',', dtype={'Total': np.int32})
df_hospitalizations.drop(columns={'Hovedstaden', 'Sjælland', 'Syddanmark', 'Midtjylland', 'Nordjylland', 'Ukendt Region'}, inplace=True)
df_hospitalizations.rename(columns={'Dato':'Date', 'Total':'Number_of_hospitalizations'}, inplace=True)
df_hospitalizations.tail(3)

,Date,Number_of_hospitalizations
439,2021-05-14,46
440,2021-05-15,32
441,2021-05-16,27


Merge Dataframes

In [8]:
df_death_hosp = pd.merge(df_deaths, df_hospitalizations, how='left', on='Date')
df_tests = pd.merge(df_tests_pcr, df_tests_antigen, how='left', on='Date', suffixes=('_pcr', '_antigen'))
df_obs = pd.merge(df_death_hosp, df_tests, how='inner', on='Date')
df_obs.fillna(value=0, inplace=True)
df_obs.head()

,Date,Number_of_deaths,Number_of_hospitalizations,NewPositive_pcr,PosPct_pcr,Tested_pcr,NewPositive_antigen,PosPct_antigen,Tested_antigen
0,2020-03-11,1,10.0,207,23.6,882,0.0,0.0,0.0
1,2020-03-12,0,18.0,44,6.6,676,0.0,0.0,0.0
2,2020-03-13,0,26.0,33,4.7,710,0.0,0.0,0.0
3,2020-03-14,1,24.0,37,6.4,583,0.0,0.0,0.0
4,2020-03-15,2,33.0,45,6.2,726,0.0,0.0,0.0


Save Dataframe in new xlsx

In [9]:
df_obs.to_excel('data/observations.xlsx')